# <center>Welcome FSP solved with local search based metaheuristics</center>

This notebook presents a practical approach to solving the flowshop problem by implementing well-known local search based metaheuristics. These metaheuristics are effective in generating high-quality solutions for large instances of the problem, requiring only a reasonable amount of computational resources. Compared to heuristics, local search metaheuristics are more effective for solving the flowshop problem because they can escape from local optima and find better solutions

# Table of Contents

1. [Johnson **n** jobs **2** machines](#Johnson-**n**-jobs-**2**-machines)
2. [CDS Heuristic](#cds-heuristic)
3. [NEH Heuristic](#neh-heuristic)
4. [Ham Heuristic](#ham-heuristic)
5. [Palmer Heuristic](#palmer-heuristic)
6. [PRSKE Heuristic](#prske-heuristic)
8. [Chen Heuristic](#Chen-heuristic-(1983))
7. [Weighted CDS Heuristic](#Weighted-CDS-heuristic)
9. [Gupta Heuristic](#gupta-heuristic)
10. [NRH Heuristic (NEW Ramzi Heuristic)](#nrh-heuristic-new-ramzi-heuristic)
11. [Kusiak Heuristic ](#Kusiak-heuristic)
12. [ Comparison Figure   ](#Comparison-Figure)




## Data utils

In [1]:
import numpy as np
import random
import time
import math
import pandas as pd
import matplotlib.pyplot as plt
from utils.benchmarks import benchmarks, upper_bound

### Path Cost calculation function :
Used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [2]:
def calculate_makespan(processing_times, sequence):
    n_jobs = len(sequence)
    n_machines = len(processing_times[0])
    end_time = [[0] * (n_machines + 1) for _ in range(n_jobs + 1)]

    for j in range(1, n_jobs + 1):
        for m in range(1, n_machines + 1):
            end_time[j][m] = max(end_time[j][m - 1], end_time[j - 1]
                                 [m]) + processing_times[sequence[j - 1]][m - 1]

    return end_time[n_jobs][n_machines]

### Gantt graph generator

In [3]:
def generate_gantt_chart(processing_times, seq, interval=50, labeled=True):
    data = processing_times.T
    nb_jobs, nb_machines = processing_times.shape
    schedules = np.zeros((nb_machines, nb_jobs), dtype=dict)
    # schedule first job alone first
    task = {"name": "job_{}".format(
        seq[0]+1), "start_time": 0, "end_time": data[0][seq[0]]}

    schedules[0][0] = task
    for m_id in range(1, nb_machines):
        start_t = schedules[m_id-1][0]["end_time"]
        end_t = start_t + data[m_id][0]
        task = {"name": "job_{}".format(
            seq[0]+1), "start_time": start_t, "end_time": end_t}
        schedules[m_id][0] = task

    for index, job_id in enumerate(seq[1::]):
        start_t = schedules[0][index]["end_time"]
        end_t = start_t + data[0][job_id]
        task = {"name": "job_{}".format(
            job_id+1), "start_time": start_t, "end_time": end_t}
        schedules[0][index+1] = task
        for m_id in range(1, nb_machines):
            start_t = max(schedules[m_id][index]["end_time"],
                          schedules[m_id-1][index+1]["end_time"])
            end_t = start_t + data[m_id][job_id]
            task = {"name": "job_{}".format(
                job_id+1), "start_time": start_t, "end_time": end_t}
            schedules[m_id][index+1] = task

    # create a new figure
    fig, ax = plt.subplots(figsize=(18, 8))

    # set y-axis ticks and labels
    y_ticks = list(range(len(schedules)))
    y_labels = [f'Machine {i+1}' for i in y_ticks]
    ax.set_yticks(y_ticks)
    ax.set_yticklabels(y_labels)

    # calculate the total time
    total_time = max([job['end_time'] for proc in schedules for job in proc])

    # set x-axis limits and ticks
    ax.set_xlim(0, total_time)
    x_ticks = list(range(0, total_time+1, interval))
    ax.set_xticks(x_ticks)

    # set grid lines
    ax.grid(True, axis='x', linestyle='--')

    # create a color dictionary to map each job to a color
    color_dict = {}
    for proc in schedules:
        for job in proc:
            if job['name'] not in color_dict:
                color_dict[job['name']] = (np.random.uniform(
                    0, 1), np.random.uniform(0, 1), np.random.uniform(0, 1))

    # plot the bars for each job on each processor
    for i, proc in enumerate(schedules):
        for job in proc:
            start = job['start_time']
            end = job['end_time']
            duration = end - start
            color = color_dict[job['name']]
            ax.barh(i, duration, left=start, height=0.5,
                    align='center', color=color, alpha=0.8)
            if labeled:
                # add job labels
                label_x = start + duration/2
                label_y = i
                ax.text(
                    label_x, label_y, job['name'][4:], ha='center', va='center', fontsize=10)

    plt.show()

### NEH HEURISTIC FOR INITIAL SOLUTION

In [4]:

def _order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [5]:
def _insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [6]:
def neh_algorithm(processing_times):
    ordered_sequence = _order_jobs_in_descending_order_of_total_completion_time(
        processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if calculate_makespan(processing_times, [
        J1, J2]) < calculate_makespan(processing_times, [J2, J1]) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = _insertion(sequence, i, job)
            Cmax_eval = calculate_makespan(processing_times, new_sequence)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

### PRSKE FOR INITIAL SOLUTION

In [7]:
def _AVG(processing_times):
    return np.mean(processing_times, axis=1)

In [8]:
def _STD(processing_times):
    return np.std(processing_times, axis=1, ddof=1)

In [9]:

def _skewness_SKE(processing_times):
    """
    Calculates the skewness of job processing times across machines.
    Return : Skewness values for each job

    """
    num_jobs, num_machines = processing_times.shape
    skewness_values = []

    for i in range(num_jobs):
        avg_processing_time = np.mean(processing_times[i, :])
        numerateur = 0
        denominateur = 0

        for j in range(num_machines):
            som = (processing_times[i, j] - avg_processing_time)
            numerateur += som ** 3
            denominateur += som ** 2

        numerateur *= (1 / num_machines)
        denominateur = (np.sqrt(denominateur * (1 / num_machines))) ** 3

        skewness_values.append(numerateur / denominateur)

    return np.array(skewness_values)

In [10]:
def PRSKE(processing_times):
    """
    Calculates the job sequence based on the PRSKE priority rule

    """
    avg = _AVG(processing_times)   # Calculate average processing times

    # Calculate standard deviation processing times
    std = _STD(processing_times)

    skw = _skewness_SKE(processing_times)  # Calculate Skewness

    order = skw + std + avg

    # Sort in descending order
    sorted_order = sorted(
        zip(order, list(range(processing_times.shape[0]))), reverse=True)

    sequence = [job for _, job in sorted_order]

    makespan = calculate_makespan(processing_times, sequence)

    return sequence,  makespan

## Neighborhood generation

### SWAP

In [11]:
def swap(solution, i, k):
    sol = solution.copy()
    sol[i], sol[k] = sol[k], sol[i]
    return sol

### Random SWAP

In [12]:
def random_swap(solution, processing_times):
    i = random.choice(list(solution))
    j = random.choice(list(solution))

    while i == j:
        j = random.choice(list(solution))

    new_solution = swap(solution, i, j)
    
    return new_solution, calculate_makespan(processing_times, new_solution)

### Best SWAP

In [13]:
def best_swap(solution, processing_times):
    sequence = solution.copy()
    num_jobs = len(solution)
    Cmax = calculate_makespan(processing_times, solution)

    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(sequence, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            if makespan < Cmax:
                sequence = new_solution
                Cmax = makespan

    return sequence, Cmax

### First Admissible SWAP

In [14]:
def first_admissible_swap(solution, processing_times):

    num_jobs = len(solution)
    Cmax = calculate_makespan(processing_times, solution)

    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(solution, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            if makespan < Cmax:
                return new_solution, makespan

    return solution, Cmax 

### First and Best Admissible SWAP

In [15]:
def fba_swap(solution, processing_times, best_global_sol):
    sequence = solution.copy()
    num_jobs = len(sequence)
    Cmax = calculate_makespan(processing_times, sequence)
    Smax = calculate_makespan(processing_times, best_global_sol)
    for i in range(num_jobs):
        for j in range(i+1, num_jobs):
            new_solution = swap(solution, i, j)
            makespan = calculate_makespan(processing_times, new_solution)

            # First improving solution
            if makespan < Cmax:
                # Improves the global solution
                if makespan < Smax:
                    return new_solution, makespan, new_solution 
                Cmax = makespan
                sequence = new_solution                

    return sequence, Cmax, best_global_sol 

## Local Based Metaheuristics

### Simulated annealing (RS) 

In [16]:
def get_neighbor(processing_times, solution, method='random_swap'):
    if method == 'random_swap':
        sol, val = random_swap(solution, processing_times)
        global_sol = sol
    elif method == 'best_swap':
        sol, val = best_swap(solution, processing_times)
        global_sol = sol
    elif method == 'first_admissible_swap':
        sol, val = first_admissible_swap(solution, processing_times)
        global_sol = sol
    elif method == 'fba_swap':
        # Returns 3 params !!!
        sol, val, global_sol = fba_swap(solution, processing_times)
    else:
        i = random.randint(0, 3)
        if i == 0:
            sol, val = random_swap(solution, processing_times)
            global_sol = sol
        elif i == 1:
            sol, val = best_swap(solution, processing_times)
            global_sol = sol
        elif i == 2:
            sol, val = first_admissible_swap(solution, processing_times)
            global_sol = sol
        else:
            sol, val, global_sol = fba_swap(solution, processing_times)
    return sol, val, global_sol

In [17]:
def RS_random(processing_times, initial_solution, temp, alpha=0.6, nb_palier= 100, it_max=1000):
    solution = initial_solution.copy()
    # print('solution',solution)
    makespan = calculate_makespan(processing_times, solution)
    # print('makespan', makespan)
    it = 0
    while it < it_max:
        for i in range(nb_palier):
            sol, value, _ = get_neighbor(processing_times, solution, method='random_swap')
            # print('Random swap sol',sol,'Its value is', value )
            delta = makespan - value
            if delta > 0:
                solution = sol
                makespan = value
                # print("I'm here")
            else:
                if random.uniform(0, 1) < math.exp(delta / temp):
                    solution = sol
                    # print("I'm here 2")
        temp = alpha * temp
        it += 1
    
    return solution

## Tests

In [18]:
matrix_data = [
    [93, 99, 82, 99, 6, 12, 90, 100, 61, 44],
    [96, 14, 108, 91, 111, 60, 67, 94, 74, 17],
    [69, 46, 68, 63, 51, 41, 34, 32, 9, 106],
    [80, 68, 32, 119, 54, 73, 14, 16, 48, 46],
    [43, 43, 33, 88, 80, 58, 102, 35, 99, 30],
    [104, 54, 13, 23, 41, 21, 87, 112, 5, 15],
    [57, 108, 76, 103, 96, 70, 57, 7, 48, 40],
    [28, 99, 114, 116, 119, 42, 24, 83, 105, 103],
    [63, 94, 30, 42, 56, 28, 78, 47, 91, 109],
    [60, 25, 22, 40, 53, 22, 30, 38, 79, 49],
    [87, 47, 94, 83, 58, 11, 42, 71, 16, 77],
    [88, 75, 75, 98, 22, 8, 73, 14, 99, 59],
    [32, 97, 97, 55, 64, 65, 75, 11, 47, 75],
    [94, 28, 51, 9, 69, 101, 79, 72, 97, 105],
    [23, 35, 9, 88, 82, 15, 83, 73, 95, 73]
]

matrix_np = np.array(matrix_data)

In [21]:
from utils.benchmarks import benchmarks
initialSolution, makespan = neh_algorithm(benchmarks[0])
rs_solution = RS_random(benchmarks[0], initialSolution, 10)




KeyboardInterrupt: 

In [ ]:
print(f'Results of NEH:')
print(f'First sequence is {initialSolution} with a makespan of {makespan}.')
print(f'Results of RS:')
print(f'Best solution {rs_solution}')



Results of NEH:
First sequence is [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19] with a makespan of 1334.
Results of RS:
Best solution [8, 14, 16, 5, 2, 18, 7, 13, 17, 15, 0, 4, 3, 1, 12, 6, 10, 9, 19, 11]


In [ ]:
calculate_makespan(benchmarks[0], rs_solution)

1278

In [18]:
initialSolution, makespan  = PRSKE(benchmarks[0])
rs_solution = RS_random(benchmarks[0], initialSolution, 10)


In [ ]:
print(f'Results of PRSKE:')
print(f'First sequence is {initialSolution} with a makespan of {makespan}.')
print(f'Results of RS:')
print(f'Best solution {rs_solution}')
calculate_makespan(benchmarks[0], rs_solution)


Results of PRSKE:


NameError: name 'initialSolution' is not defined